# Experimentación

En este notebook corremos los experimentos descritos en el informe

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import subprocess

def run(algorithm: str, instance_path: str) -> (int, float):
    """Corre el programa para la instancia dada y devuelve (resultado, tiempo de ejecucion)."""
    result = subprocess.run(
        f"../build/npm {algorithm} < {instance_path}",
        shell=True, capture_output=True, text=True, check=True,
    )

    return int(result.stdout), float(result.stderr)

In [3]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def run_solapamiento(instance_path: str):
    """Corre el programa para la instancia dada y devuelve (resultado, matriz de solapamiento)."""
    result = subprocess.run(
        f"../build/npm GR < {instance_path}",
        shell=True, capture_output=True, text=True, check=True,
    )

    return json.loads(result.stderr.split('\n')[0])["pd_accesses"]


# TODO: arreglar sta mierda
#df1 = pd.DataFrame(run_solapamiento("instancias/grupos/1-grupos_1.in"))
#df25 = pd.DataFrame(run_solapamiento("instancias/grupos/25-grupos_1.in"))

#fig, ax = plt.subplots(ncols=2, sharey=True)
#g = sns.heatmap(data=df25, cbar=True, ax=ax[0])
#g.set_title("25")
#sns.heatmap(data=df1, cbar=True, ax=ax[1])

In [4]:
from typing import List

def get_instances(dataset: str) -> List[str]:
    """Para cada dataset devuelve la lista de instancias correspondientes con el path completo"""
    instances = os.listdir(f"instancias/{dataset}")
    return list(filter(
        lambda i: i.endswith(".in"),
        map(lambda i: f"instancias/{dataset}/"+i, instances),
    ))


In [5]:
def run_instance(rows: list, dataset: str, algorithms: List[str]):
    """
    Corre una instancia para todos los algoritmos, llenando por referencia
    la lista de rows. Lanza una excepcion si para alguna instancia los
    resultados de todos los algoritmos no coinciden.
    """
    for instance in tqdm(get_instances(dataset), "instancias"):
        results = {}
        for alg in algorithms:
            res, t = run(alg, instance)
            rows.append({
                "dataset": dataset,
                "algorithm": alg,
                "time_ms": t,
                "instance": instance,
            })

            results[alg] = res

        if not np.alltrue([results[algorithms[0]] == res for res in results.values()]):
            print(f"Ojo que con la instancia {instance} no dieron todos iguales. Resultados: {results}")

## Implementación de la experimentación

In [9]:
ALL = ["FB", "BT", "BT-F", "BT-O-G", "BT-O-C", "DP"]

In [11]:
# type experimento struct {
#     algoritmos: []string
#     dataset: string
# }

experimentos = [
    {
        "algoritmos" : ALL,
        "dataset"    : "control",
    },
    {
    # DP para ver cómo afecta el solapamiento y BT debería ser siempre más o menos igual
    # TODO: revisar la matriz de solapamiento
        "algoritmos" : ["BT", "DP"],
        "dataset"    : "grupos",
    },
    # Optimalidad
    {
        "algoritmos" : ALL,
        "dataset"    : "one-to-rule",
    },
    {
        "algoritmos" : ALL,
        "dataset"    : "identicos",
    },
    # Factibilidad
    {
        "algoritmos" : ALL,
        "dataset"    : "low-M",
    },
    # Programación Dinámica
    {
        "algoritmos" : ["BT-F", "DP"],
        "dataset"    : "cache",
    },
    {
        "algoritmos" : ["DP"],
        "dataset"    : "complejidad-DP",
    }
]


In [13]:
import os
import numpy as np
import pandas as pd

from tqdm import tqdm

rows = []

for i, experimento in enumerate(experimentos):
    print("Corriendo el experimento {0} ({1}/{2})".format(experimento["dataset"], i+1, len(experimentos)))
    run_instance(rows, experimento["dataset"], experimento["algoritmos"])

df_results = pd.DataFrame(rows, columns=["dataset", "algorithm", "time_ms", "instance"])
print("Finished!")

instancias:   4%|▍         | 3/78 [00:00<00:03, 22.79it/s]

Corriendo el experimento control (1/7)
Ojo que con la instancia instancias/control/n_19-i_1.in no dieron todos iguales. Resultados: {'FB': 430, 'BT': 399, 'BT-F': 430, 'BT-O-G': 399, 'BT-O-C': 430, 'DP': 430}


instancias:   6%|▋         | 5/78 [00:00<00:03, 20.23it/s]

Ojo que con la instancia instancias/control/n_20-i_0.in no dieron todos iguales. Resultados: {'FB': 356, 'BT': 325, 'BT-F': 356, 'BT-O-G': 325, 'BT-O-C': 356, 'DP': 356}


instancias:  12%|█▏        | 9/78 [00:00<00:07,  9.24it/s]

Ojo que con la instancia instancias/control/n_18-i_0.in no dieron todos iguales. Resultados: {'FB': 441, 'BT': 441, 'BT-F': 441, 'BT-O-G': 363, 'BT-O-C': 441, 'DP': 441}


instancias:  14%|█▍        | 11/78 [00:02<00:15,  4.26it/s]


KeyboardInterrupt: 

In [9]:
df_results.to_csv("resultados-lowm.csv")